In [1]:
### prepare receptor (protien) in pdbqt format
### obabel is stand along execution
### write a .bat script to covert protein.pdb to protein.pdbqt
### change the IN_pdb and OUT_pdbqt file names

code="""
@echo off
REM ====== Set paths ======
set IN_pdb=tkd_variants_pdb/p.Cys797Ser_aligned.pdb
set OUT_pdbqt=receptor/p.Cys797Ser_clean_h.pdbqt

REM ====== Run Open Babel example (convert ligand if needed) ======
obabel %IN_pdb% -O %OUT_pdbqt% -p 7.4 --partialcharge eem -xr -xp -xn

echo Coverting protein.pdb to protein.pdbqt
pause
"""
# Save to file
with open("pdb2pdbqt.bat", "w") as f:
    f.write(code)

In [2]:
## executiing covertion protein.pdb to protein.pdbqt
import subprocess
import os

if not os.path.exists("receptor"):
    os.makedirs("receptor")
result = subprocess.run(["cmd", "/c", "pdb2pdbqt.bat"], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Coverting protein.pdb to protein.pdbqt
Press any key to continue . . . 

*** Open Babel Warning  in OpenBabel::OBMol::PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is tkd_variants_pdb/p.Cys797Ser_aligned.pdb)

1 molecule converted



In [3]:
### preparation ligand list from ChEMBL2363049_subset
import pandas as pd
import requests

# EGFR ChEMBL target ID
df = pd.read_csv('CHEMBL_dataset/CHEMBL2363049_subset.csv',sep=';')
#print(df)
#print(df.info())
df['Standard Type'].value_counts()

df_ic50=df[df['Standard Type']=='IC50']
df_ic50 = df_ic50.dropna(subset=['Standard Value', 'Standard Units'])
#print(df_ic50)
# Save to CSV
df_ic50.rename(columns={'Smiles': 'smiles'}, inplace=True)
df_ic50.to_csv("chembl_egfr_subset_ic50.csv", index=False)


In [46]:
import argparse
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdDistGeom, MolToSmiles, MolFromSmiles, AddHs
from meeko import MoleculePreparation
from meeko import PDBQTWriterLegacy
#import vina
#from vina import Vina
from itertools import *
import sys, os, glob, re
import multiprocessing, subprocess
from multiprocessing import Process, Pool

outfolder = 'data/p.Cys797Ser/'
receptor = 'receptor/p.Cys797Ser_clean_h.pdbqt'
inputdata = 'chembl_egfr_subset_ic50.csv'
vina_exe_path = r"vina_1.2.7_win.exe"
maps_size_angstroms = [15, 15, 15]
numPoses = 5 
    
if not os.path.exists(outfolder):
    os.makedirs(outfolder)
dataload = pd.read_csv(inputdata)

assert 'smiles' in dataload.columns
assert 'Molecular Weight' in dataload.columns
assert 'Molecule ChEMBL ID' in dataload.columns

#print(dataload['smiles']) ## smiles column could be smiles or other molecule files
ligands=dataload[['Molecule ChEMBL ID','smiles','Molecular Weight']]
ligands = ligands[:4]  ## reduce the list for demonstration
print(ligands)

  Molecule ChEMBL ID                                             smiles  \
0      CHEMBL1923013  OCCNCCn1ccc2ncnc(Nc3ccc(Oc4cccc(C(F)(F)F)c4)c(...   
1      CHEMBL1923020  CN(CCn1ccc2ncnc(Nc3ccc(Oc4cccc(C(F)(F)F)c4)c(C...   
2      CHEMBL1923011  OCCOCCn1ccc2ncnc(Nc3ccc(Oc4cccc(OC(F)(F)F)c4)c...   
3      CHEMBL1921812         Clc1cccc(Oc2ccc(Nc3ncnc4cc[nH]c34)cc2Cl)c1   

   Molecular Weight  
0            491.90  
1            519.91  
2            508.88  
3            371.23  


In [47]:
def read_input_mol(ligand_descriptions,keep_local_structures=False):
    ligands_list = []
    index_list = []
    failed_ligand_indices = []
    print('Reading molecules and generating local structures with RDKit')
    for idx, ligand in enumerate(ligand_descriptions['smiles']):
        try:
            mol = MolFromSmiles(ligand)  # check if it is a smiles or a path
            if mol is not None:
                mol = AddHs(mol)
                generate_conformer(mol)
                ligands_list.append(mol)
                index_list.append(idx)
            else:
                ### read_molecule support mol2, sdf, pdbqt, pdb inputs
                mol = read_molecule(ligand, remove_hs=False, sanitize=True)
                if mol is None:
                    raise Exception('RDKit could not read the molecule ', ligand)
                if not keep_local_structures:
                    mol.RemoveAllConformers()
                    mol = AddHs(mol)
                    generate_conformer(mol)
                ligands_list.append(mol)
                index_list.append(idx)
        except Exception as e:
            print('Failed to read molecule ', ligand, ' We are skipping it. The reason is the exception: ', e)
            failed_ligand_indices.append(idx)
            continue


    # put index, descriptions, mol in a dict
    ligand = dict()
    headers = ['idx','Molecule ChEMBL ID','description','mol','Molecular Weight']
    ligand[headers[0]]=index_list
    ligand[headers[1]]=ligand_descriptions['Molecule ChEMBL ID'].tolist()
    ligand[headers[2]]=ligand_descriptions['smiles'].tolist()
    ligand[headers[3]]=ligands_list
    ligand[headers[4]]=ligand_descriptions['Molecular Weight'].tolist()

    return ligand


def read_molecule(molecule_file, sanitize=False, calc_charges=False, remove_hs=False):
    if molecule_file.endswith('.mol2'):
        mol = Chem.MolFromMol2File(molecule_file, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.sdf'):
        supplier = Chem.SDMolSupplier(molecule_file, sanitize=False, removeHs=False)
        mol = supplier[0]
    elif molecule_file.endswith('.pdbqt'):
        with open(molecule_file) as file:
            pdbqt_data = file.readlines()
        pdb_block = ''
        for line in pdbqt_data:
            pdb_block += '{}\n'.format(line[:66])
        mol = Chem.MolFromPDBBlock(pdb_block, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.pdb'):
        mol = Chem.MolFromPDBFile(molecule_file, sanitize=False, removeHs=False)
    else:
        raise ValueError('Expect the format of the molecule_file to be '
                         'one of .mol2, .sdf, .pdbqt and .pdb, got {}'.format(molecule_file))

    try:
        if sanitize or calc_charges:
            Chem.SanitizeMol(mol)

        if calc_charges:
            # Compute Gasteiger charges on the molecule.
            try:
                AllChem.ComputeGasteigerCharges(mol)
            except:
                warnings.warn('Unable to compute charges for the molecule.')

        if remove_hs:
            mol = Chem.RemoveHs(mol, sanitize=sanitize)
    except Exception as e:
        print(e)
        print("RDKit was unable to read the molecule.")
        return None

    return mol


def generate_conformer(mol):
    ps = AllChem.ETKDGv2()
    # id = AllChem.EmbedMolecule(mol, ps)
    for repeat in range(50):
        rid = AllChem.EmbedMolecule(mol, ps)
        if rid == 0:
            break
    if rid == -1:
        print('rdkit coords could not be generated without using random coords. using random coords now.')
        ps.useRandomCoords = True
        AllChem.EmbedMolecule(mol, ps)
        AllChem.MMFFOptimizeMolecule(mol, confId=0)

    AllChem.MMFFOptimizeMolecule(mol, mmffVariant='MMFF94s', maxIters=500)

def print_ligand_center(molsetup):
    """Compute center of mass for a Meeko MoleculeSetup (RDKit-based)."""
    conf = molsetup.mol.GetConformer()
    coords = []
    masses = []
    for atom in molsetup.mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        coords.append([pos.x, pos.y, pos.z])
        masses.append(atom.GetMass())
    coords = np.array(coords)
    masses = np.array(masses)
    com = (coords * masses[:, None]).sum(axis=0) / masses.sum()
    return com.tolist()

        
def get_ligand_com_from_pdbqt(pdbqt_file):
    coords = []
    with open(pdbqt_file, 'r') as f:
        for line in f:
            if line.startswith("HETATM") or line.startswith("ATOM"):
                x = float(line[30:38])
                y = float(line[38:46])
                z = float(line[46:54])
                coords.append([x, y, z])
    coords = np.array(coords)
    center = coords.mean(axis=0)
    return center.tolist()


In [48]:
ligands_mol = read_input_mol(ligands)

Reading molecules and generating local structures with RDKit


In [49]:
def run_single_autodock(ligands_mol, receptor, maps_size_angstroms):
    # use single CPU cores
    for i in range(len(ligands_mol)):
        try:
            #ini_energy_min=dock_vina(args, idx, lig)
            ini_energy_min=dock_vina_win(ligands_mol['idx'][i],
                                         ligands_mol['mol'][i],
                                         ligands_mol['Molecular Weight'][i],
                                         receptor,
                                         vina_exe_path,
                                         maps_size_angstroms,
                                         numPoses
                                        )

        except KeyboardInterrupt:
            print('Keyboard interrupt detected.')
            sys.exit(0)

In [9]:
def dock_vina_win(idx, lig, mw, receptor,vina_exe_path, maps_size_angstroms, numPoses):
    """
    args: has attributes receptor (path), outfolder (folder), lignames, molweight, maps_size_angstroms, numPoses
    idx: index of ligand
    lig: ligand object or path (should be converted to pdbqt file before calling this function)
    vina_path: full path or command to vina_win.exe
    """

    # Step 1: Prepare ligand pdbqt file path
    ligprep = MoleculePreparation()
    lig_setup = ligprep.prepare(lig)
    for setup in lig_setup:
        lig_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok and lig_string is not None:
            pdbqt_path = os.path.join(outfolder, f"ligand_{idx}.pdbqt")  # set your desired file path
            with open(pdbqt_path, 'w') as f:
                f.write(lig_string)
        else:
            print("Error in generating ligand pdbqt string:", error_msg)
            sys.exit(0)
            
        vina_minimization_dock(idx, ligprep, mw, receptor, vina_exe_path)
        vina_multiple_dock(idx, receptor, vina_exe_path , maps_size_angstroms, numPoses)

def convert_single_model_pdbqt(input_file, output_file):
    """Convert a single-model multi-MODEL PDBQT to plain PDBQT format."""
    with open(input_file, "r") as infile:
        lines = infile.readlines()

    new_lines = []
    inside_model = False
    for line in lines:
        if line.startswith("MODEL"):
            inside_model = True
            continue
        elif line.startswith("ENDMDL"):
            inside_model = False
            continue
        else:
            new_lines.append(line)

    with open(output_file, "w") as outfile:
        outfile.writelines(new_lines)

    #print(f"Converted {input_file} → {output_file} (MODEL tags removed).")   


def vina_minimization_dock(idx,ligprep,mw, receptor,vina_exe_path):
    # Step 1: Prepare ligand pdbqt file path
    ligand_pdbqt = os.path.join(outfolder, f"ligand_{idx}.pdbqt")
    # If `lig` is an object that needs conversion, convert and write pdbqt outside this function
    # Here we assume ligand_pdbqt exists

    # Step 2: Calculate docking box size based on MW (same logic as your original)
    if mw < 450:
        maps_size = [20, 20, 20]
    elif mw < 550:
        maps_size = [25, 25, 25]
    else:
        maps_size = [30, 30, 30]

    print(f'ligand {idx} MW: {mw} initial box size: {maps_size}')

    # Step 3: Calculate center of ligand for docking box (you must implement this function or adapt)
    center=print_ligand_center(ligprep.setup)
    print('ligand_center: ',center)
    #center = print_ligand_center(lig)  # returns [x,y,z]

    # Step 4: Prepare output docking file
    vina_minimized_out = os.path.join(outfolder, f"ligand_{idx}_vina_minimized.pdbqt")
    #Build vina_win.exe command line
    # Step 5: minimization step
    cmd = [
        vina_exe_path,
        "--receptor", receptor,
        "--ligand", ligand_pdbqt,
        "--center_x", str(center[0]),
        "--center_y", str(center[1]),
        "--center_z", str(center[2]),
        "--size_x", str(maps_size[0]),
        "--size_y", str(maps_size[1]),
        "--size_z", str(maps_size[2]),
        "--out", vina_minimized_out,
        "--exhaustiveness", "8",
        "--num_modes", "1",
    ]

    print("Running Vina with command:")
    print(" ".join(cmd))

    # Step 6: Run docking minimization subprocess
    result = subprocess.run(cmd, capture_output=True, text=True)

    print("Vina Minimization STDOUT:")
    print(result.stdout)
    print("Vina Minimization STDERR:")
    print(result.stderr)

    if result.returncode != 0:
        raise RuntimeError(f"Vina docking minimization failed with exit code {result.returncode}")
    
    return vina_minimized_out

def vina_multiple_dock(idx, receptor, vina_path, maps_size_angstroms, numPoses):
    convert_single_model_pdbqt(os.path.join(outfolder,f"ligand_{idx}_vina_minimized.pdbqt"), os.path.join(outfolder,f"ligand_{idx}_vina_minimized_clean.pdbqt"))

    vina_out = os.path.join(outfolder, f"ligand_{idx}_vina.pdbqt")
    center_minimized=get_ligand_com_from_pdbqt(os.path.join(outfolder,f"ligand_{idx}_vina_minimized.pdbqt"))
    print('center_minimized: ',center_minimized)
    ligand_pdbqt = os.path.join(outfolder, f"ligand_{idx}_vina_minimized_clean.pdbqt")

    # Step 7: docking step

    cmd = [
        vina_exe_path,
        "--receptor", receptor,
        "--ligand", ligand_pdbqt,
        "--center_x", str(center_minimized[0]),
        "--center_y", str(center_minimized[1]),
        "--center_z", str(center_minimized[2]),
        "--size_x", str(maps_size_angstroms[0]),
        "--size_y", str(maps_size_angstroms[1]),
        "--size_z", str(maps_size_angstroms[2]),
        "--out", vina_out,
        "--exhaustiveness", "32",
        "--num_modes", str(numPoses),
    ]

    print("Running Vina with command:")
    print(" ".join(cmd))

    # Step 8: Run docking subprocess
    result = subprocess.run(cmd, capture_output=True, text=True)

    print("Vina STDOUT:")
    print(result.stdout)
    print("Vina STDERR:")
    print(result.stderr)

    if result.returncode != 0:
        raise RuntimeError(f"Vina docking failed with exit code {result.returncode}")    
    
    # Optionally: parse docking score from vina output if needed
    # For now, just return the output path

    return vina_out

In [10]:
run_single_autodock(ligands_mol, receptor, maps_size_angstroms)

ligand 0 MW: 491.9 initial box size: [25, 25, 25]
ligand_center:  [0.9506075025141296, 0.1642311904105097, 0.10801433821177388]
Running Vina with command:
vina_1.2.7_win.exe --receptor receptor/p.Cys797Ser_clean_h.pdbqt --ligand outfolder\ligand_0.pdbqt --center_x 0.9506075025141296 --center_y 0.1642311904105097 --center_z 0.10801433821177388 --size_x 25 --size_y 25 --size_z 25 --out outfolder\ligand_0_vina_minimized.pdbqt --exhaustiveness 8 --num_modes 1


C:\Users\mkhsi\test\Lib\site-packages\meeko\preparation.py:420: DeprecationWarning: MoleculePreparation.setup is deprecated in Meeko v0.5. MoleculePreparation.prepare() returns a list of MoleculeSetup instances.
  warnings.warn(msg, DeprecationWarning)


Vina Minimization STDOUT:
AutoDock Vina v1.2.7
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.c

In [50]:
def collecpposes(ligands_mol):
    filePaths = glob.glob(outfolder+'/*_vina.pdbqt')
    posesDict = dict()
    #print(filePaths)
    
    for filePath in filePaths:
        #print(filePath)
        # Get the interaction residues
        matches = re.search(r'ligand_(\d+)_vina.pdbqt',filePath)
        if not matches:
            continue
        # Get residue indices
        idx = int(matches.groups()[0])

        # Read in the first line (header) output file and count number of total lines.
        f = open(filePath,'r')
        lines = f.readlines()

        # Ignore lines not starting with ENERGY:
        lines = [line for line in lines if line.startswith('REMARK VINA RESULT:')]
        f.close()
        
        lines = [line.strip('\n').split() for line in lines if line.startswith('REMARK VINA RESULT:')]
        ## every model has two energy results and the second entry is repeated for all poses 
        ## if no further minimization changes the intramolecular geometry.
        lines = lines[::2]
        
        lines = [[float(integer) for integer in line[3:]] for line in lines]
        #print(f'num vian result: {len(lines)}')
        #print(f'line: {lines}')
        ## if docking models are less than numPoses, increase  --exhaustiveness
        ## somehow in vina c
        headers = ['affinity','RMSD_lb','RMSD_ub']
        headerColumns = [0,1,2,3] 

        numTitles = len(headers)
        # Assign each column into appropriate key's value in energyOutput dict.
        poseOutput = dict()
        for i in range(0,numTitles):
            poseOutput[headers[i]] = [line[headerColumns[i]] for line in lines]
        #print(f'poseOutput: {poseOutput}')
        
        # add descfiption to the dictonary
        for i in range(len(ligands_mol['idx'])):
            if ligands_mol['idx'][i] == idx:
                poseOutput['description']=ligands_mol['description'][i]

        # Puts this energyOutput dict into energies dict with keys as residue ids
        posesDict[str(idx)] = poseOutput
        #print(posesDict)
        
        
    # Prepare a pandas data table from parsed energies, write it to new files depending on type of energy
    df_affinity = pd.DataFrame()
    df_description = pd.DataFrame()

    for key,value in list(posesDict.items()):
        # Try-check block to allow collecting even when parse of pair IE fails.
        try:
            df_affinity[key] = value['affinity']
            df_description[key] = [value['description']]

        except:
            print('Failed to parse interaction data for pair '+str(key))

    tmp_df = df_affinity.transpose().sort_index()
    tmp_df.insert(loc=0, column='ligand',value=df_description.transpose().iloc[:,0])
    print(tmp_df)
    tmp_df.to_csv(os.path.join(outfolder,'docking_affinity.csv'))
    
    return None

In [51]:
collecpposes(ligands_mol)

                                              ligand      0      1      2  \
0  OCCNCCn1ccc2ncnc(Nc3ccc(Oc4cccc(C(F)(F)F)c4)c(... -8.040 -7.960 -7.915   
1  CN(CCn1ccc2ncnc(Nc3ccc(Oc4cccc(C(F)(F)F)c4)c(C... -8.629 -8.505 -8.472   
2  OCCOCCn1ccc2ncnc(Nc3ccc(Oc4cccc(OC(F)(F)F)c4)c... -7.427 -7.364 -7.237   
3         Clc1cccc(Oc2ccc(Nc3ncnc4cc[nH]c34)cc2Cl)c1 -8.489 -8.455 -8.363   

       3      4  
0 -7.914 -7.835  
1 -8.471 -8.365  
2 -7.163 -7.120  
3 -8.283 -8.038  
